# Import my personal printing object

In [5]:
from parameters import ParameterContainer, ParameterColumn, ParameterTable

# Import and name constants

In [6]:
import numpy as np
from scipy import constants
c = constants.physical_constants["speed of light in vacuum"][0] #m/sec by default
e = constants.physical_constants["elementary charge"][0] #m/sec by default
m_e_c2 = constants.physical_constants["electron mass energy equivalent in MeV"][0]*10**-3 #in GeV
r_e = constants.physical_constants["classical electron radius"][0]#in m
h_bar = constants.physical_constants["Planck constant over 2 pi in eV s"][0] #in eV s
alpha = constants.physical_constants["fine-structure constant"][0] #no units

# Name and assign input parameters

In [7]:
column1 = ParameterContainer()
column1.addParameter("E",1.5,"GeV")
column1.addParameter("B",1.,"T")

column2 = ParameterContainer()
column2.addParameter("E",1.5,"GeV")
column2.addParameter("B",3.,"T")

column3 = ParameterContainer()
column3.addParameter("E",3.,"GeV")
column3.addParameter("B",3.,"T")

table = ParameterTable()
table.addColumn(ParameterColumn(column1))
table.addColumn(ParameterColumn(column2))
table.addColumn(ParameterColumn(column3))

# Calculate and store derived parameters

In [8]:
for column in table:
    electron_energy = column.header_parameters.getParameter("E").value()
    derived_parameters = ParameterContainer()
    electron_kinetic_energy = derived_parameters.addParameter("electron kinetic energy",(electron_energy - m_e_c2)*10**9,"eV").value()
    derived_parameters.addParameter("electron momentum",np.sqrt(electron_energy**2 - m_e_c2**2)*10**9/c,"eV/c")
    derived_parameters.addParameter("electron gamma",electron_kinetic_energy/(m_e_c2*10**9) + 1)
    if electron_energy == 1.5:
        electron_charge = derived_parameters.addParameter("beam charge",10e-12,"C").value()
    elif electron_energy == 3.0:
        electron_charge = derived_parameters.addParameter("beam charge",7e-12,"C").value()
    else:
        raise Exception("You need to pick an electron charge for your pulse.")
    derived_parameters.addParameter("number electrons",electron_charge/e)
    column.associateParameterContainer("Derived",derived_parameters)

table.printTable("Derived")

|  | E = 1.5 GeV,B = 1.0 T | E = 1.5 GeV,B = 3.0 T | E = 3.0 GeV,B = 3.0 T |

| :-: | :-: | :-: | :-: |

| electron kinetic energy (eV) | 1499489001.05 | 1499489001.05 | 2999489001.05 |

| electron momentum (eV/c) | 5.00346113764 | 5.00346113764 | 10.0069227108 |

| electron gamma | 2935.42679774 | 2935.42679774 | 5870.85359548 |

| beam charge (C) | 1e-11 | 1e-11 | 7e-12 |

| number electrons | 62415091.2588 | 62415091.2588 | 43690563.8812 |

# Ring/turn parameters storing and reporting

In [9]:
for column in table:
    electron_energy = column.header_parameters.getParameter("E").value()
    quad_B = column.header_parameters.getParameter("B").value()
    electron_momentum = column.parameter_containers["Derived"].getParameter("electron momentum").value()
    electron_gamma = column.parameter_containers["Derived"].getParameter("electron gamma").value()
    
    ring_parameters = ParameterContainer()
    rho = ring_parameters.addParameter("rho",electron_momentum/quad_B,"m").value()
    L_bend = ring_parameters.addParameter("length of bend",np.pi/2.*rho,"m")
    U0 = ring_parameters.addParameter("synchrotron energy loss per turn",4.*np.pi/3.*r_e*electron_gamma**4/rho*m_e_c2*10**6,"keV")
    E_photon = ring_parameters.addParameter("energy of sychrotron emitted photon",h_bar*3./2.*c*electron_gamma**3/rho/1000,"keV")
    column.associateParameterContainer("Ring",ring_parameters)
    
table.printTable("Ring")


|  | E = 1.5 GeV,B = 1.0 T | E = 1.5 GeV,B = 3.0 T | E = 3.0 GeV,B = 3.0 T |

| :-: | :-: | :-: | :-: |

| rho (m) | 5.00346113764 | 1.66782037921 | 3.33564090359 |

| length of bend (m) | 7.85941837626 | 2.61980612542 | 5.23961247887 |

| synchrotron energy loss per turn (keV) | 89.506565483 | 268.519696449 | 2148.1574781 |

| energy of sychrotron emitted photon (keV) | 1.49630786462 | 4.48892359385 | 17.955693594 |

# Doublet parameters storing and reporting 

In [10]:
G1 = 50. #Magnetic field gradient for the first doublet quad in T/m.
G2 = 100. #Magnetic field gradient for the second doublet quad in T/m.
L_quad = 0.10 #Length of the doublet quads in m.

for column in table:
    electron_energy = column.header_parameters.getParameter("E").value()
    quad_B = column.header_parameters.getParameter("B").value()
    electron_momentum = column.parameter_containers["Derived"].getParameter("electron momentum").value()
    electron_gamma = column.parameter_containers["Derived"].getParameter("electron gamma").value()
    
    doublet_parameters = ParameterContainer()
    doublet_parameters.addParameter("Quad length",L_quad,"m")
    doublet_parameters.addParameter("Quad gradient 1",G1,"T/m")
    doublet_parameters.addParameter("Quad gradient 2",G2,"T/m")
    K1 = doublet_parameters.addParameter("K1",G1/electron_momentum,"1/m^2").value()
    K2 = doublet_parameters.addParameter("K2",G2/electron_momentum,"1/m^2").value()
    f1 = doublet_parameters.addParameter("f1",electron_momentum/(G1*L_quad),"m").value()
    f2 = doublet_parameters.addParameter("f2",electron_momentum/(G2*L_quad),"m").value()
    d = doublet_parameters.addParameter("Post quad drift",np.sqrt(f2**2 * f1 / (f1-f2)),"m").value()
    quad_spacing = doublet_parameters.addParameter("Quad spacing",f2 * f1 / d,"m").value()
    doublet_parameters.addParameter("Doublet focus",d + quad_spacing,"m")
    
    column.associateParameterContainer("Doublet",doublet_parameters)
    
table.printTable("Doublet")

|  | E = 1.5 GeV,B = 1.0 T | E = 1.5 GeV,B = 3.0 T | E = 3.0 GeV,B = 3.0 T |

| :-: | :-: | :-: | :-: |

| Quad length (m) | 0.1 | 0.1 | 0.1 |

| Quad gradient 1 (T/m) | 50.0 | 50.0 | 50.0 |

| Quad gradient 2 (T/m) | 100.0 | 100.0 | 100.0 |

...

| f1 (m) | 1.00069222753 | 1.00069222753 | 2.00138454216 |

| f2 (m) | 0.500346113764 | 0.500346113764 | 1.00069227108 |

| Post quad drift (m) | 0.707596259965 | 0.707596259965 | 1.41519258152 |

| Quad spacing (m) | 0.707596259965 | 0.707596259965 | 1.41519258152 |

| Doublet focus (m) | 1.41519251993 | 1.41519251993 | 2.83038516304 |

# Laser parameters

In [11]:
for column in table:
    laser_parameters = ParameterContainer()
    laser_wavelength = laser_parameters.addParameter("wavelength",780,"nm").value()
    if electron_energy == 1.5:
        a_0 = laser_parameters.addParameter("a_0",2.1).value()
    elif electron_energy == 3.0:
        a_0 = laser_parameters.addParameter("a_0",2.5).value()
    else:
        raise Exception("You need to pick an a_0 for your laser.")
    intensity = laser_parameters.addParameter("intensity",a_0**2/(laser_wavelength*1e-3)**2/7.3e-19,"W/cm^2").value()

    column.associateParameterContainer("Laser",laser_parameters)
    
table.printTable("Laser")

|  | E = 1.5 GeV,B = 1.0 T | E = 1.5 GeV,B = 3.0 T | E = 3.0 GeV,B = 3.0 T |

| :-: | :-: | :-: | :-: |

| wavelength (nm) | 780 | 780 | 780 |

| a_0 | 2.5 | 2.5 | 2.5 |

| intensity (W/cm^2) | 1.40723928922e+19 | 1.40723928922e+19 | 1.40723928922e+19 |

# Plasma parameters

In [12]:
for column in table:
    a_0 = column.parameter_containers["Laser"].getParameter("a_0").value()
    
    plasma_parameters = ParameterContainer()
    if electron_energy == 1.5:
        n = plasma_parameters.addParameter("plasma density",3e17,"cm^-3").value()
        E_max = plasma_parameters.addParameter("Max E field",np.sqrt(n * a_0**3)/1e9,"GV/cm").value()
    elif electron_energy == 3.0:
        n = plasma_parameters.addParameter("plasma density",1.75e17,"cm^-3").value()
        E_max = plasma_parameters.addParameter("Max E field",np.sqrt(n * a_0**2)/1e9,"GV/cm").value()
    else:
        raise Exception("You need to pick an a_0 for your laser.")
    
    
    plasma_wavelength = plasma_parameters.addParameter("plasma wavelength",0.1*np.sqrt(1e17/n),"mm").value()
    w_pe = plasma_parameters.addParameter("plasma frequency",2.*np.pi*c/(plasma_wavelength*1e-3),"rad/sec").value()
    R_b = plasma_parameters.addParameter("bubble size",2*np.sqrt(a_0)*c/w_pe,"m").value()
    laser_power = plasma_parameters.addParameter("total laser power",np.pi*R_b**2*intensity*100.**2/1e12,"TW").value()
    L_p = plasma_parameters.addParameter("depletion length",0.5 / a_0 * plasma_wavelength**3 / (laser_wavelength*1e-6)**2/10,"cm").value()
    L_d = plasma_parameters.addParameter("dephasing length",L_p*a_0/10,"cm").value()
    max_energy = plasma_parameters.addParameter("maximum energy possible",E_max*np.min([L_p,L_d]),"GeV").value()
    total_plasma_length = plasma_parameters.addParameter("total plasma length",3. / E_max,"cm").value()

    column.associateParameterContainer("Plasma",plasma_parameters)
    
table.printTable("Plasma")

|  | E = 1.5 GeV,B = 1.0 T | E = 1.5 GeV,B = 3.0 T | E = 3.0 GeV,B = 3.0 T |

| :-: | :-: | :-: | :-: |

| plasma density (cm^-3) | 1.75e+17 | 1.75e+17 | 1.75e+17 |

| Max E field (GV/cm) | 1.04582503317 | 1.04582503317 | 1.04582503317 |

| plasma wavelength (mm) | 0.0755928946018 | 0.0755928946018 | 0.0755928946018 |

...

| total laser power (TW) | 639.911682836 | 639.911682836 | 639.911682836 |

| depletion length (cm) | 14.1998487089 | 14.1998487089 | 14.1998487089 |

| dephasing length (cm) | 3.54996217723 | 3.54996217723 | 3.54996217723 |

| maximum energy possible (GeV) | 3.71263931174 | 3.71263931174 | 3.71263931174 |

| total plasma length (cm) | 2.8685486624 | 2.8685486624 | 2.8685486624 |

In [13]:
plasma_parameters_2 = ParameterContainer()
n = plasma_parameters_2.addParameter("plasma density",1.75e17,"cm^-3").value()
E_max = plasma_parameters_2.addParameter("Max E field",np.sqrt(n * a_0**2)/1e9,"GV/cm").value()
plasma_wavelength = plasma_parameters_2.addParameter("plasma wavelength",0.1*np.sqrt(1e17/n),"mm").value()
w_pe = plasma_parameters_2.addParameter("plasma frequency",2.*np.pi*c/(plasma_wavelength*1e-3),"rad/sec").value()
R_b = plasma_parameters_2.addParameter("bubble size",2*np.sqrt(a_0)*c/w_pe,"m").value()
laser_power = plasma_parameters_2.addParameter("total laser power",np.pi*R_b**2*intensity*100.**2/1e12,"TW").value()
L_p = plasma_parameters_2.addParameter("depletion length",0.5 / a_0 * plasma_wavelength**3 / (laser_wavelength*1e-6)**2/10,"cm").value()
L_d = plasma_parameters_2.addParameter("dephasing length",L_p*a_0/10,"cm").value()
max_energy = plasma_parameters_2.addParameter("maximum energy possible",E_max*np.min([L_p,L_d]),"GeV").value()
total_plasma_length = plasma_parameters_2.addParameter("total plasma length",3. / E_max,"cm").value()
critical_power = total_plasma_length = plasma_parameters_2.addParameter("critical power",21.5*( a_0*R_b/(laser_wavelength*1e-9) )**2/1000,"TW").value()

print plasma_parameters_2

plasma density = 1.75e+17 cm^-3
Max E field = 1.04582503317 GV/cm
plasma wavelength = 0.0755928946018 mm
plasma frequency = 2.4918368019e+13 rad/sec
bubble size = 3.80453082601e-05 m
total laser power = 639.911682836 TW
depletion length = 14.1998487089 cm
dephasing length = 3.54996217723 cm
maximum energy possible = 3.71263931174 GeV
total plasma length = 2.8685486624 cm
critical power = 319.691792334 TW


# Wiggler parameters

In [14]:
for column in table:
    number_electrons = column.parameter_containers["Derived"].getParameter("number electrons").value()
    wiggler_parameters = ParameterContainer()

    if electron_energy == 1.0:
        photon_energy = wiggler_parameters.addParameter("photon energy",0.4,"keV").value()
        wiggler_wavelength = wiggler_parameters.addParameter("wiggler periodicity",1.5e-2,"m").value()
    elif electron_energy == 3.0:
        photon_energy = wiggler_parameters.addParameter("photon energy",10.,"keV").value()
        wiggler_wavelength = wiggler_parameters.addParameter("wiggler periodicity",1.e-1,"m").value()
    else:
        raise Exception("You need to pick an arbitrary photon energy for your electon energy.")
    
    wiggler_rho = wiggler_parameters.addParameter("rho",h_bar*1.5*c*electron_gamma**3/(photon_energy*1e3),"m").value()
    wiggler_B = wiggler_parameters.addParameter("magnetic_field",3.33* electron_momentum / wiggler_rho,"T").value()
    K = wiggler_parameters.addParameter("K",electron_gamma*wiggler_wavelength/wiggler_rho,"").value()
    photons_per_unit_length_per_electron = wiggler_parameters.addParameter("photons per m per e",alpha*electron_gamma/wiggler_rho,"1/(m*electron)").value()
    photons_per_unit_length = wiggler_parameters.addParameter("photons per m",photons_per_unit_length_per_electron*number_electrons,"1/m").value()
    energy_loss_per_m_per_electron = wiggler_parameters.addParameter("energy per m",photons_per_unit_length_per_electron*photon_energy,"keV/m").value()

    column.associateParameterContainer("Wiggler",wiggler_parameters)
    
table.printTable("Wiggler")

|  | E = 1.5 GeV,B = 1.0 T | E = 1.5 GeV,B = 3.0 T | E = 3.0 GeV,B = 3.0 T |

| :-: | :-: | :-: | :-: |

| photon energy (keV) | 10.0 | 10.0 | 10.0 |

| wiggler periodicity (m) | 0.1 | 0.1 | 0.1 |

| rho (m) | 5.98937460044 | 5.98937460044 | 5.98937460044 |

| magnetic_field (T) | 5.56369485129 | 5.56369485129 | 5.56369485129 |

| K () | 98.0211455643 | 98.0211455643 | 98.0211455643 |

| photons per m per e (1/(m*electron)) | 7.15294858145 | 7.15294858145 | 7.15294858145 |

| photons per m (1/m) | 446451938.481 | 446451938.481 | 312516356.937 |

| energy per m (keV/m) | 71.5294858145 | 71.5294858145 | 71.5294858145 |

In [15]:
'{:^14}'.format(1)

'      1       '

In [16]:
width = 32
string_format = '{:^'+str(width)+'}'
string_format.format(1)

'               1                '

In [17]:
"String %s and float %.3f" % ("string thing", 3.54398187)

'String string thing and float 3.544'

Tables | Are | Cool |
------------- |:-------------:| -----:|
col 3 is | right-aligned | $1600 |
col 2 is | centered |$12 |
zebra stripes | are neat | $1 |